# A

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

#variables
animation_history = []
dots = [(.5, 1.4), (2.3, 1.9), (2.9, 3.2)]

graph_x_list = []
curve_y_list = []

lines_y_list = []

#functions
def get_curve(line_slope = .64):
    x_values = np.linspace(0, 2)
    y_values = []

    for x in x_values:
        y = 0

        for current_dot in dots:
            dot_x = current_dot[0]
            dot_y = current_dot[1]
            y += (dot_y - (x + (line_slope * dot_x))) ** 2
        
        y_values.append(y)
    
    return (x_values, y_values)


def get_tan(intercept, slope, line_slope = .64):
    y_values = []

    y0 = 0 #error

    for current_dot in dots:
        dot_x = current_dot[0]
        dot_y = current_dot[1]
        y0 += (dot_y - (intercept + (line_slope * dot_x))) ** 2


    for x0 in graph_x_list:
        y = y0 + slope * (x0 - intercept)
        y_values.append(y)
        
    lines_y_list.append(y_values)
    animation_history.append([intercept, y0])


def slope_calculator(intercept, line_slope):
    slope = 0
    for current_dot in dots:
        x = current_dot[0]
        y = current_dot[1]
        slope += -2 * (y - (intercept + (line_slope * x)))
    
    return slope


def gradient_descend(learning_rate = .1, step_size = 0, line_slope = .64, intercept = 0, minimum_step_size = .001, max_number_of_steps = 1000):
    step_count = 0
    while abs(step_size) > minimum_step_size or step_count == 0:
        if step_count > max_number_of_steps:
            break

        else:
            step_count += 1
            print(f"\n---Loop {step_count}---")

        #getting the curve slope
        slope = slope_calculator(intercept, line_slope)
        print(f"Curve slope: {slope}")

        #getting the tangent
        get_tan(intercept, slope)

        #updating the step size
        step_size = slope * learning_rate
        print(f"Step size: {step_size}")
        
        #updating he intercept
        print(f"Old intercept: {intercept}")

        intercept -= step_size
        print(f"New intercept: {intercept}")
    
    return step_count


#program
(graph_x_list, curve_y_list) = get_curve(.64)
step_count = gradient_descend(learning_rate=.1)

#plot
fig, graph = plt.subplots(figsize=(7, 7))
plt.close()

point_marker, = graph.plot([], [], 'o', color='blue')
tangent_line, = graph.plot([], [], '-', color='red')

graph.set_xlim(0, 2)
graph.set_ylim(0, 4)
graph.set_xticks([i * .25 for i in range(0, 9)])
graph.set_yticks([i * .25 for i in range(0, 17)])
graph.grid(True)
graph.set_xlabel("Intercept", fontsize=12)
graph.set_ylabel("Error", fontsize=12)
graph.plot(graph_x_list, curve_y_list)

def update_animation(frame):
    history = animation_history[frame]
    intercept, error = history

    point_marker.set_data([intercept], [error])
    tangent_line.set_data(graph_x_list, lines_y_list[frame])

    return point_marker, tangent_line

anim = FuncAnimation(fig, update_animation, frames=step_count, interval=100, blit=True)
HTML(anim.to_jshtml())



---Loop 1---
Curve slope: -5.704000000000001
Step size: -0.5704000000000001
Old intercept: 0
New intercept: 0.5704000000000001

---Loop 2---
Curve slope: -2.281599999999999
Step size: -0.22815999999999992
Old intercept: 0.5704000000000001
New intercept: 0.79856

---Loop 3---
Curve slope: -0.9126399999999997
Step size: -0.09126399999999997
Old intercept: 0.79856
New intercept: 0.8898240000000001

---Loop 4---
Curve slope: -0.3650560000000005
Step size: -0.03650560000000005
Old intercept: 0.8898240000000001
New intercept: 0.9263296000000001

---Loop 5---
Curve slope: -0.14602239999999966
Step size: -0.014602239999999968
Old intercept: 0.9263296000000001
New intercept: 0.9409318400000001

---Loop 6---
Curve slope: -0.05840895999999951
Step size: -0.005840895999999951
Old intercept: 0.9409318400000001
New intercept: 0.946772736

---Loop 7---
Curve slope: -0.02336358400000016
Step size: -0.002336358400000016
Old intercept: 0.946772736
New intercept: 0.9491090944

---Loop 8---
Curve slope: 

ValueError: shape mismatch: objects cannot be broadcast to a single shape.  Mismatch is between arg 0 with shape (50,) and arg 1 with shape (0,).